# Fluent Python, Chapter 2, An Array of Sequence

**Container sequences**: list, tuple, and collections.deque can hold items of different types. Hold references to the objects they contain, which may be of any type

**Flat sequences**: str, bytes, bytearray, memoryview, and array.array hold items of one type. physically store the value of each item within its own memory space, and not as distinct objects. Thus, flat sequences are more compact, but they are limited to holding primitive values like characters, bytes, and numbers.


## List Comprehensions (listcomps) and Generator Experession (genexps)

A for loop may be used to do lots of different things: scanning a sequence to count or pick items, computing aggregates (sums, averages), or any number of other processing tasks.

In contrast, a listcomp is meant to do one thing only: **to build a new list**. If the list comprehension spans more than two lines, it is probably best to break it apart or rewrite as a plain old for loop.

To fill up other sequence types (tuples, arrays, other types), a genexp is the way to go. It saves memory because it yields items **one by one** using the iterator protocol instead of building a whole list just to feed another constructor.

In [7]:
symbols = '$¢£¥€¤'
codes = [ord(symbol) for symbol in symbols if ord(symbol) > 127] # this is list comprehension
codes

[194, 162, 194, 163, 194, 165, 226, 130, 172, 194, 164]

In [10]:
# this is generator expressions, 
# just need one parentheses if just function is single argument
tuple(ord(symbol) for symbol in symbols) 

(36, 194, 162, 194, 163, 194, 165, 226, 130, 172, 194, 164)

In [11]:
import array
array.array('I', (ord(symbol) for symbol in symbols)) # we have to use 2 parentheseses 

array('I', [36L, 194L, 162L, 194L, 163L, 194L, 165L, 226L, 130L, 172L, 194L, 164L])

In [13]:
colors = ['black', 'white']
sizes = ['S', 'M', 'L']
for tshirt in ('%s %s' % (c, s) for c in colors for s in sizes): # use this way is more preferable than list comprehension
    print(tshirt)

black S
black M
black L
white S
white M
white L


## Tuples

Tuples do double duty: they can be used as **immutable lists** and also as **records** with no field names

In [14]:
city, year, pop, chg, area = ('Tokyo', 2003, 32450, 0.66, 8014) # Tuple Unpacking

In [15]:
latitude, longitude = 33.9425, -118.408056 # parallel assignment
latitude, longitude = longitude, latitude # swapping the values of variables without using a temporary variable 

In [16]:
# prefixing an argument with a star when calling a function
divmod(20, 8)
t = (20, 8)
divmod(*t)

(2, 4)

In [17]:
_, remainder = divmod(*t) #_ is used as placeholder, we don't care about its value

### Using * to graph excess items in python 3 only

    a, b, *rest = range(5)
    a, b, rest 
    (0, 1, [2, 3, 4])
    
    a, b, *rest = range(3)
    a, b, rest 
    (0, 1, [2])
    
    a, b, *rest = range(2)
    a, b, rest 
    (0, 1, [])

    a, *body, c, d = range(5)
    a, body, c, d
    (0, [1, 2], 3, 4)
    
    *head, b, c, d = range(5)
    head, b, c, d
    ([0, 1], 2, 3, 4)

In [20]:
metro_areas = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)), 
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)), 
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)), 
    ('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
]

print('{:15} | {:^9} | {:^9}'.format('', 'lat.', 'long.')) 
fmt = '{:15} | {:9.4f} | {:9.4f}' 
for name, cc, pop, (latitude, longitude) in metro_areas:
    if longitude <= 0: 
        print(fmt.format(name, latitude, longitude))

                |   lat.    |   long.  
Mexico City     |   19.4333 |  -99.1333
New York-Newark |   40.8086 |  -74.0204
Sao Paulo       |  -23.5478 |  -46.6358


### Named Tuples

The collections.namedtuple function is a factory that produces subclasses of tuple enhanced with **field names** and a class name—which helps debugging.

In [21]:
from collections import namedtuple
City = namedtuple('City', 'name country population coordinates')
tokyo = City('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
tokyo

City(name='Tokyo', country='JP', population=36.933, coordinates=(35.689722, 139.691667))

In [22]:
tokyo.population

36.933

In [23]:
tokyo.coordinates

(35.689722, 139.691667)

In [25]:
tokyo[1]

'JP'

tuple supports all list methods that do not involve adding or removing items, with one exception—tuple lacks the __re versed__ method. However, that is just for optimization; reversed(my_tuple) works without it.

In [26]:
l = [1, 2, 3]
id(l)

4355805768

## When a List is not the Answer

### Arrays
If the list will only contain numbers, an array.array is more efficient than a list. It supports all mutable sequence operations (including .pop, .insert, and .extend), and additional methods for fast loading and saving such as .frombytes and .tofile.

In [29]:
>>> from array import array

>>> from random import random

>>> floats = array('d', (random() for i in range(10**7)))

>>> floats[-1]

0.07802343889111107

>>> fp = open('floats.bin', 'wb')

>>> floats.tofile(fp) # write to file

>>> fp.close()

>>> floats2 = array('d')

>>> fp = open('floats.bin', 'rb') # read from file

>>> floats2.fromfile(fp, 10**7)

>>> fp.close()

>>> floats2[-1]

0.19826694863197847

In [30]:
%timeit floats2[-1]

The slowest run took 50.55 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 61.3 ns per loop


In [31]:
floats2 == floats

True

- If we store other types of data, use pickle instead.
- if we want to use advanced array and matrix operations, use NumPy and SciPy instead

### Deques and Other Queues

The .append and .pop methods make a list usable as a stack or a queue (if you use .append and .pop(0), you get LIFO behavior). But inserting and removing from the left of a list (the 0-index end) is costly because the entire list must be shifted. 

The class collections.deque is a thread-safe double-ended queue designed for fast inserting and removing from both ends. It is also the way to go if you need to keep a list of “last seen items” or something like that, because a deque can be bounded—i.e., created with a maximum length—and then, when it is full, it discards items from the opposite end when you append new ones.

In [32]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!
